# Q1: Scalar Computation Graph and Gradient

## TODO

In [0]:
def g_w1(x1: float, x2: float, w1: float, w2: float) -> float:
    g = 0
    raise NotImplementedError
    return g

def g_w2(x1: float, x2: float, w1: float, w2: float) -> float:
    g = 0
    raise NotImplementedError
    return g

def g_x1(x1: float, x2: float, w1: float, w2: float) -> float:
    g = 0
    raise NotImplementedError
    return g

def g_x2(x1: float, x2: float, w1: float, w2: float) -> float:
    g = 0
    raise NotImplementedError
    return g

In [0]:
x1 = 0.2
x2 = 0.4
w1 = 0.3
w2 = -0.5

grads_x1 = []
grads_x2 = []
grads_w1 = []
grads_w2 = []
w1s = [w1]
w2s = [w2]

for i in range(30):
    grad_w1 = g_w1(x1, x2, w1, w2)    
    grad_w2 = g_w2(x1, x2, w1, w2)
    
    w2 += -0.01 * grad_w2
    w1 += -0.01 * grad_w1
    
    w1s.append(w1)
    w2s.append(w2)

    grads_w1.append(grad_w1)
    grads_w2.append(grad_w2)

# Q3: Neural Network From Scratch

In this section, we will implement a neural network to solve MNIST, a hand-written digits (0-9) classification dataset.

## Set up

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from typing import Dict
from functools import partial

from tqdm import tqdm
tqdm.monitor_interval = 0
tqdm = partial(tqdm, bar_format='{l_bar}{r_bar}')

import torch
import torch.nn.functional as F
import torchvision

In [0]:
TRAIN_SIZE = 50_000

train = torchvision.datasets.MNIST('./data', train=True, transform=None, target_transform=None, download=True)
test = torchvision.datasets.MNIST('./data', train=False, transform=None, target_transform=None, download=True)

train_x = train.data.float().numpy()
train_y = train.targets.numpy()

shuffle_idx = np.arange(len(train_x))
np.random.RandomState(0).shuffle(shuffle_idx)
train_x = train_x[shuffle_idx]
train_y = train_y[shuffle_idx]

dev_x, dev_y = train_x[TRAIN_SIZE:], train_y[TRAIN_SIZE:]
train_x, train_y = train_x[:TRAIN_SIZE], train_y[:TRAIN_SIZE]

test_x = test.data.float().numpy()
test_y = test.targets.numpy()

Sample of images



In [0]:
i = 4048 #@param {type: "slider", min: 0, max: 10000}
print(f'label = {train_y[i]}')
plt.imshow(train_x[i])
plt.show()

Each images have the same shape of 28-by-28. We flatten the matrix and pretend it is just one long vector.

In [0]:
NB_FEAT = 28 * 28

Normalize the feature. Note that we only use the training set to compute the mean and standard derivation.

In [0]:
mean = train_x.mean()
std = train_x.std()

train_x = (train_x - mean) / (std + 1e-7)
dev_x = (dev_x - mean) / (std + 1e-7)
test_x = (test_x - mean) / (std + 1e-7)

train_x = train_x.reshape(-1, NB_FEAT)
dev_x = dev_x.reshape(-1, NB_FEAT)
test_x = test_x.reshape(-1, NB_FEAT)

In [0]:
def row_logsumexp(x):
    # numerical stablization
    x_max = x.max(axis=1).reshape(-1, 1)
    return x_max + np.log(np.exp(x - x_max).sum(axis=1)).reshape(-1, 1)


In [0]:
class Parameters:
    def __init__(self):
        self.param: Dict[str, np.ndarray] = {}
        self.grad: Dict[str, np.ndarray] = {}

    def set_param(self, key: str, param: np.ndarray):
        self.param[key] = param
        self.grad[key] = np.zeros_like(param)

    def get_param(self, key):
        assert key in self.param, f'variable {key} is not part of the Parameter'
        return self.param[key]

    def accumlate_grad(self, key: str, grad: np.ndarray):
        assert key in self.param, f'variable {key} is not part of the Parameter'
        assert self.param[key].shape == grad.shape, f'for variable {key}, the shape of parameter and the shape of gradient is not matched'
        self.grad[key] += grad

    def zero_grad(self):
        for key in self.param:
            self.grad[key] = np.zeros_like(self.param[key])

    def apply_grad(self, lr: float):
        for key in self.param.keys():
            assert self.param[key].shape == self.grad[key].shape, f'for variable {key}, the shape of parameter and the shape of gradient is not matched'
            self.param[key] -= self.grad[key] * lr

In [0]:
def init_linear(input_dim, output_dim):
    return np.random.RandomState(0).randn(input_dim, output_dim) * np.sqrt(2 / input_dim)

In [0]:
def main_training(params: Parameters, forward_and_backward, train_x, train_y, dev_x, dev_y, batch_size, learning_rate, nb_epochs):
    assert isinstance(params, Parameters)
    step = 0
    best_acc = 0
    for epoch in tqdm(range(nb_epochs)):
        train_loss = []
        lr = learning_rate
        for i in range(0, len(train_x), batch_size):
            batch_x = train_x[i:i+batch_size]
            batch_y = train_y[i:i+batch_size]

            loss, probs = forward_and_backward(batch_x, batch_y, params)
            params.apply_grad(lr)

            train_loss.append(loss)
            step += 1

        accs = []
        for i in range(0, len(dev_x), batch_size):
            batch_x = dev_x[i:i+batch_size]
            batch_y = dev_y[i:i+batch_size]

            loss, probs = forward_and_backward(batch_x, batch_y, params, grad=False)
            pred_y = probs.argmax(axis=1)
            accs.append(pred_y == batch_y)
        acc = np.mean(accs) * 100
        print(f'epoch {epoch} train loss = {np.mean(train_loss):.3f} dev accuracy = {acc:.2f}%')
        if acc > best_acc:
            best_acc = acc

    print(f'best dev accuracy = {best_acc:.2f}%')

## Q2.1 Linear Classifier

Let $x \in R^{1\times N}$ be a single sample. $N$ is the number of the feature dimension and $C$ is the number of class in the final classification. Note $N=28*28$ and $C=10$

In the softmax layer, $W_{sm} \in R^{N \times C}$. The unnormalized score $s \in R^{C}$ is

$$s = x\cdot W_{sm}$$

With softmax function, we get the probability $p(c_i)$ of class $i$ where $i \in \{1,\cdots,C\}$

$$p(c_i) = \frac {\exp(s_i)} {\sum_i \exp(s_i)}$$

where $s_i$ is the $i^{th}$ element of $s$.

Assuming $y \in \{1,\cdots,C\}$, we want to minimize $-\log p(c_y)$, the cross-entropy loss between one hot true distribution `q(c_i) = 1 if i == y else 0` and $p(c_i)$.

We will take the average $-\log p(c_y)$ of a batch of x as the loss.

In `linear_classifier_forward_and_backward`, `batch_x` and `batch_y` are matrixs, where each row are $x$ and $y$, respectively.

In [0]:
linear_classifier = Parameters()
linear_classifier.set_param('w_sm', init_linear(NB_FEAT, 10))

### TODO

In [0]:
def linear_classifier_forward_and_backward(batch_x, batch_y, param: Parameters, grad=True) -> (float, np.ndarray):
    '''
    compute the loss and the gradient of each parameter
    return loss (float) and probs p(c_i) (matrix with shape batch_size x C)
    '''
    param.zero_grad()
    w_sm = param.get_param('w_sm')

    # compute loss and probs
    loss = 0
    probs = np.zeros((len(batch_x), 10))
    raise NotImplementedError

    if not grad:
        return loss, probs

    # compute gradient
    w_sm_grad = np.zeros_like(w_sm)
    raise NotImplementedError

    # save the gradient
    param.accumlate_grad('w_sm', w_sm_grad)
    return loss, probs

### Loss & Gradient Check

If all assertions are passed, it means you are good to go.

In [0]:
# use a prime number for batch size would make debugging easier
bs = 7
batch_x = train_x[:bs]
batch_y = train_y[:bs]
loss, _ = linear_classifier_forward_and_backward(batch_x, batch_y, linear_classifier)

w_sm = linear_classifier.get_param('w_sm')
_w_sm = torch.tensor(w_sm, dtype=torch.double, requires_grad=True)
_batch_x = torch.tensor(batch_x, dtype=torch.double, requires_grad=False)
_batch_y = torch.tensor(batch_y, dtype=torch.long, requires_grad=False)

_logits = F.linear(_batch_x, _w_sm.T)
_logprobs = F.log_softmax(_logits, dim=-1)
_loss = -_logprobs[torch.arange(len(_batch_y)), _batch_y].mean()
_loss.backward()

assert np.isclose(loss, _loss.item())
assert np.allclose(_w_sm.grad.numpy(), linear_classifier.grad['w_sm'])

### Training

In [0]:
BS = 50
LR = 0.005
NB_EPOCH = 20

linear_classifier = Parameters()
linear_classifier.set_param('w_sm', init_linear(NB_FEAT, 10))

main_training(linear_classifier, linear_classifier_forward_and_backward, train_x, train_y, dev_x, dev_y, BS, LR, NB_EPOCH)

## Q2.1 MLP with Single Hidden Layer

Let $x \in R^{1\times N}$ be a single sample. $N$ is the number of the feature dimension and $C$ is the number of class in the final classification.

For the first layer, $W_1 \in R^{N\times \frac{N}{2}}$ and $b_1 \in R^{\frac{N}{2}}$

$$\bar{h}_1 = x\cdot W_1 + b_1$$

$$h_1 = ReLU(\bar{h}_1)$$

In the softmax layer, $W_{sm} \in R^{\frac{N}{2} \times C}$. The unnormalized score $s \in R^{C}$ is

$$s = h_1\cdot W_{sm}$$

With softmax function, we get the probability $p(c_i)$ of class $i$ where $i \in \{1,\cdots,C\}$

$$p(c_i) = \frac {\exp(s_i)} {\sum_i \exp(s_i)}$$

where $s_i$ is the $i^{th}$ element of $s$.

Assuming $y \in \{1,\cdots,C\}$, we want to minimize $-\log p(c_y)$, the cross-entropy loss between one hot true distribution `q(c_i) = 1 if i == y else 0` and $p(c_i)$.

We will take the average $-\log p(c_y)$ of a batch of x as the loss.

In `mlp_single_hidden_forward_and_backward`, `batch_x` and `batch_y` are matrixs, where each row are $x$ and $y$, respectively.

In [0]:
mlp_single_hidden = Parameters()
mlp_single_hidden.set_param('w1', init_linear(NB_FEAT, NB_FEAT // 2))
mlp_single_hidden.set_param('b1', np.zeros(NB_FEAT // 2))
mlp_single_hidden.set_param('w_sm', init_linear(NB_FEAT // 2, 10))

### TODO



In [0]:
def mlp_single_hidden_forward_and_backward(batch_x, batch_y, param: Parameters, grad=True):
    '''
    compute the loss and the gradient of each parameter
    return loss (float) and p(c_i) (matrix with shape batch_size x C)
    '''
    param.zero_grad()
    w1 = param.get_param('w1')
    b1 = param.get_param('b1')
    w_sm = param.get_param('w_sm')


    # compute loss and probs
    loss = 0
    probs = np.zeros((len(batch_x), 10))
    raise NotImplementedError

    if not grad:
        return loss, probs

    # compute gradient
    w1_grad = np.zeros_like(w1)
    b1_grad = np.zeros_like(b1)
    w_sm_grad = np.zeros_like(w_sm)
    raise NotImplementedError

    param.accumlate_grad('w1', w1_grad)
    param.accumlate_grad('b1', b1_grad)
    param.accumlate_grad('w_sm', w_sm_grad)
    return loss, probs

### Loss & Gradient Check

If all assertions are passed, it means you are good to go.

In [0]:
# use a prime number for batch size would make debugging easier
bs = 7
batch_x = train_x[:bs]
batch_y = train_y[:bs]
loss, _ = mlp_single_hidden_forward_and_backward(batch_x, batch_y, mlp_single_hidden)

w1 = mlp_single_hidden.get_param('w1')
b1 = mlp_single_hidden.get_param('b1')
w_sm = mlp_single_hidden.get_param('w_sm')
_w1 = torch.tensor(w1, dtype=torch.double, requires_grad=True)
_b1 = torch.tensor(b1, dtype=torch.double, requires_grad=True)
_w_sm = torch.tensor(w_sm, dtype=torch.double, requires_grad=True)
_batch_x = torch.tensor(batch_x, dtype=torch.double, requires_grad=False)
_batch_y = torch.tensor(batch_y, dtype=torch.long, requires_grad=False)

_h1 = F.linear(_batch_x, _w1.T) + _b1
_h1 = F.relu(_h1)
_logits = F.linear(_h1, _w_sm.T)
_logprobs = F.log_softmax(_logits, dim=-1)
_loss = -_logprobs[torch.arange(len(_batch_y)), _batch_y].mean()
_loss.backward()

assert np.isclose(loss, _loss.item())
assert np.allclose(_w_sm.grad.numpy(), mlp_single_hidden.grad['w_sm'])
assert np.allclose(_w1.grad.numpy(), mlp_single_hidden.grad['w1'])
assert np.allclose(_b1.grad.numpy(), mlp_single_hidden.grad['b1'])

### Train!

In [0]:
BS = 50
LR = 0.005
NB_EPOCH = 20

mlp_single_hidden = Parameters()
mlp_single_hidden.set_param('w1', init_linear(NB_FEAT, NB_FEAT // 2))
mlp_single_hidden.set_param('b1', np.zeros(NB_FEAT // 2))
mlp_single_hidden.set_param('w_sm', init_linear(NB_FEAT // 2, 10))

main_training(mlp_single_hidden, mlp_single_hidden_forward_and_backward, train_x, train_y, dev_x, dev_y, BS, LR, NB_EPOCH)

## Q2.3 MLP with Two Hidden Layer

Let $x \in R^{1\times N}$ be a single sample. $N$ is the number of the feature dimension and $C$ is the number of class in the final classification.

For the first layer, $W_1 \in R^{N\times \frac{N}{2}}$ and $b_1 \in R^{\frac{N}{2}}$

$$\bar{h}_1 = x\cdot W_1 + b_1$$

$$h_1 = ReLU(\bar{h}_1)$$

For the second layer, $W_2 \in R^{\frac{N}{2} \times \frac{N}{2}}$ and $b_2 \in R^{\frac{N}{2}}$

$$\bar{h}_2 = h_1\cdot W_2 + b_2$$

$$h_2 = h_1 + ReLU(\bar{h}_2)$$

*Sidenote: in deep learning, $g(x, f) = x + f(x)$ is usually referred to as skip connection.*

In the softmax layer, $W_{sm} \in R^{\frac{N}{2} \times C}$. The unnormalized score $s \in R^{C}$ is

$$s = h_2\cdot W_{sm}$$

With softmax function, we get the probability $p(c_i)$ of class $i$ where $i \in \{1,\cdots,C\}$

$$p(c_i) = \frac {\exp(s_i)} {\sum_i \exp(s_i)}$$

where $s_i$ is the $i^{th}$ element of $s$.

Assuming $y \in \{1,\cdots,C\}$, we want to minimize $-\log p(c_y)$, the cross-entropy loss between one hot true distribution `q(c_i) = 1 if i == y else 0` and $p(c_i)$.

We will take the average $-\log p(c_y)$ of a batch of x as the loss.

In `mlp_two_hidden_forward_and_backward`, `batch_x` and `batch_y` are matrixs, where each row are $x$ and $y$, respectively.

In [0]:
mlp_two_hidden = Parameters()
mlp_two_hidden.set_param('w1', init_linear(NB_FEAT, NB_FEAT // 2))
mlp_two_hidden.set_param('b1', np.zeros(NB_FEAT // 2))
mlp_two_hidden.set_param('w2', init_linear(NB_FEAT // 2, NB_FEAT // 2))
mlp_two_hidden.set_param('b2', np.zeros(NB_FEAT // 2))
mlp_two_hidden.set_param('w_sm', init_linear(NB_FEAT // 2, 10))

### TODO


In [0]:
def mlp_two_hidden_forward_and_backward(batch_x, batch_y, param: Parameters, grad=True):
    '''
    compute the loss and the gradient of each parameter
    return loss (float) and p(c_i) (matrix with shape batch_size x C)
    '''
    param.zero_grad()
    w1 = param.get_param('w1')
    b1 = param.get_param('b1')
    w2 = param.get_param('w2')
    b2 = param.get_param('b2')
    w_sm = param.get_param('w_sm')

    # compute loss and probs
    loss = 0
    probs = np.zeros((len(batch_x), 10))
    raise NotImplementedError

    if not grad:
        return loss, probs

    # compute gradient
    w1_grad = np.zeros_like(w1)
    b1_grad = np.zeros_like(b1)
    w2_grad = np.zeros_like(w2)
    b2_grad = np.zeros_like(b2)
    w_sm_grad = np.zeros_like(w_sm)
    raise NotImplementedError

    param.accumlate_grad('w1', w1_grad)
    param.accumlate_grad('b1', b1_grad)
    param.accumlate_grad('w2', w2_grad)
    param.accumlate_grad('b2', b2_grad)
    param.accumlate_grad('w_sm', w_sm_grad)
    return loss, probs


### Loss & Gradient Check

If all assertions are passed, it means you are good to go.

In [0]:
# use a prime number for batch size would make debugging easier
bs = 7
batch_x = train_x[:bs]
batch_y = train_y[:bs]
loss, _ = mlp_two_hidden_forward_and_backward(batch_x, batch_y, mlp_two_hidden)

w1 = mlp_two_hidden.get_param('w1')
b1 = mlp_two_hidden.get_param('b1')
w2 = mlp_two_hidden.get_param('w2')
b2 = mlp_two_hidden.get_param('b2')
w_sm = mlp_two_hidden.get_param('w_sm')
_w1 = torch.tensor(w1, dtype=torch.double, requires_grad=True)
_b1 = torch.tensor(b1, dtype=torch.double, requires_grad=True)
_w2 = torch.tensor(w2, dtype=torch.double, requires_grad=True)
_b2 = torch.tensor(b2, dtype=torch.double, requires_grad=True)
_w_sm = torch.tensor(w_sm, dtype=torch.double, requires_grad=True)
_batch_x = torch.tensor(batch_x, dtype=torch.double, requires_grad=False)
_batch_y = torch.tensor(batch_y, dtype=torch.long, requires_grad=False)

_h1 = F.linear(_batch_x, _w1.T) + _b1
_h1 = F.relu(_h1)

_h2 = F.linear(_h1, _w2.T) + _b2
_h2 = F.relu(_h2)
_h2 = _h2 + _h1

_logits = F.linear(_h2, _w_sm.T)
_logprobs = F.log_softmax(_logits, dim=-1)
_loss = -_logprobs[torch.arange(len(_batch_y)), _batch_y].mean()
_loss.backward()

assert np.isclose(loss, _loss.item())
assert np.allclose(_w_sm.grad.numpy(), mlp_two_hidden.grad['w_sm'])
assert np.allclose(_w2.grad.numpy(), mlp_two_hidden.grad['w2'])
assert np.allclose(_b2.grad.numpy(), mlp_two_hidden.grad['b2'])
assert np.allclose(_w1.grad.numpy(), mlp_two_hidden.grad['w1'])
assert np.allclose(_b1.grad.numpy(), mlp_two_hidden.grad['b1'])

### Train!

In [0]:
BS = 50
LR = 0.005
NB_EPOCH = 20

mlp_two_hidden = Parameters()
mlp_two_hidden.set_param('w1', init_linear(NB_FEAT, NB_FEAT // 2))
mlp_two_hidden.set_param('b1', np.zeros(NB_FEAT // 2))
mlp_two_hidden.set_param('w2', init_linear(NB_FEAT // 2, NB_FEAT // 2))
mlp_two_hidden.set_param('b2', np.zeros(NB_FEAT // 2))
mlp_two_hidden.set_param('w_sm', init_linear(NB_FEAT // 2, 10))

main_training(mlp_two_hidden, mlp_two_hidden_forward_and_backward, train_x, train_y, dev_x, dev_y, BS, LR, NB_EPOCH)